In [2]:
!pip install --upgrade pip
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension install --py widgetsnbextension
!pip install jupyterlab_widgets
!pip install transformers
!pip install torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


import torch
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm
from transformers import RobertaTokenizer, RobertaModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
text = "Replace me by any text you'd like."
model = model.to(device)
encoded_input = tokenizer(text, return_tensors='pt').to(device)
output = model(**encoded_input)

# Load the transformer model and tokenizer
#model_name = "https://huggingface.co/FacebookAI/roberta-base"  
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModel.from_pretrained(model_name)
#model = model.to(device)  # Move to device (GPU or CPU)

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbclassic nbconvert notebook run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [-

/opt/conda/envs/pytorch-gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Load the dataset from the provided gzip file
import gzip

# Path to the dataset
dataset_path = "assignment4-dataset.txt.gz"

# Read dataset into a list of sentences
dataset_sentences = []
try:
    with gzip.open(dataset_path, 'rt') as file:
        for line in file:
            try:
                line = line.strip()
                if line:
                    dataset_sentences.append(line)
            except UnicodeDecodeError:
                continue
except EOFError:
    print("Warning: Reached unexpected end of file. Processed only valid lines.")

print(f"Loaded dataset with {len(dataset_sentences)} sentences.")

Loaded dataset with 583903 sentences.


In [4]:
# Tokenize a subset of the dataset for demonstration purposes
subset_sentences = dataset_sentences[:100]  # Process the first 100 sentences as a demonstration
tokenized_inputs = tokenizer(subset_sentences, padding=True, truncation=True, return_tensors="pt")
tokenized_inputs = {key: val.to(device) for key, val in tokenized_inputs.items()}  # Move tensors to the correct device
print("Tokenization complete for 100 sentences.")


Tokenization complete for 100 sentences.


In [5]:
# Pass tokenized inputs through the model

with torch.no_grad():
    outputs = model(**tokenized_inputs)
contextual_embeddings = outputs.last_hidden_state  # (batch_size, seq_len, hidden_size)
    

In [12]:

from collections import defaultdict
import numpy as np
import pickle

# Placeholder tokenizer and contextual_embeddings (replace with your actual objects)
# Assuming contextual_embeddings has shape (len(text_list), max_tokens, embedding_dim)

# Define the text list
text_list = [
    "This is the first sentence.",
    "Here is another one.",
    "Machine learning is fascinating!"
]

# Placeholder for contextual_embeddings for demonstration
# Replace with the actual embeddings from your model
max_tokens = 100
embedding_dim = 768
contextual_embeddings = np.random.rand(len(text_list), max_tokens, embedding_dim)

# Initialize token_embeddings as defaultdict
token_embeddings = defaultdict(list)

# Process each text
for i, text in enumerate(text_list):
    tokens = tokenizer.tokenize(text)  # Tokenize the text
    print(f"Processing text {i}: {text}")
    print(f"Tokens: {tokens}")
    print(f"Embedding shape for text {i}: {contextual_embeddings[i].shape}")

    for j, token in enumerate(tokens):
        # Ensure index `j` does not exceed embedding dimensions
        if j < contextual_embeddings.shape[1]:
            token_embeddings[token].append(contextual_embeddings[i, j])  # No need to move to CPU for numpy
        else:
            print(f"Token '{token}' at index {j} exceeds embedding size {contextual_embeddings.shape[1]}")

# Compute average embedding for each token
static_embeddings = {token: np.mean(embeds, axis=0) for token, embeds in token_embeddings.items()}

# Save the embeddings for later use
with open("static_embeddings.pkl", "wb") as f:
    pickle.dump(static_embeddings, f)

print(f"Generated static embeddings for {len(static_embeddings)} unique tokens.")


Processing text 0: This is the first sentence.
Tokens: ['This', 'Ġis', 'Ġthe', 'Ġfirst', 'Ġsentence', '.']
Embedding shape for text 0: (100, 768)
Processing text 1: Here is another one.
Tokens: ['Here', 'Ġis', 'Ġanother', 'Ġone', '.']
Embedding shape for text 1: (100, 768)
Processing text 2: Machine learning is fascinating!
Tokens: ['Machine', 'Ġlearning', 'Ġis', 'Ġfascinating', '!']
Embedding shape for text 2: (100, 768)
Generated static embeddings for 13 unique tokens.
